In [82]:
import json
from bs4 import BeautifulSoup
from collections import OrderedDict


# set up the corpus object
# and wtf does it being a translation suggest we do :O
# now the work
corpus = OrderedDict()
corpus["author"] = [{"name": "A. S. Kline"}, {"slug": "tony-kline"}, {"full name": "Tony Kline"}, {"abbreviation": "A. Kl."}]
works_list = []


# includes Amores, Ars Amatoria, and Remedia
html_file = "Kline_Love_Poems.html"

html_doc = open(html_file).read()
soup = BeautifulSoup(html_doc, 'html.parser')

def get_amores(works_list):
    # article tags surround works, all other tags are unnested, boo
    for index, work in enumerate(soup.findAll('article')):
        # ignore the preface material
        if index==1:
            # the h1 is the title
            work_dict = OrderedDict() # otherwise the title is last which is not what we want in the JSON
            work_dict["title"] = work.find('h1').text
#             work_dict["abbreviation"] = work.attrib["n"]
            work_dict["slug"] = "amores"
            
            
            books_list = []
            book_looping = True
            # only h2s with this class indicate Book titles
            for index, book in enumerate(work.findAll('h2', class_='text-align-center'), start=1):
                book_dict = OrderedDict()
                book_dict["book_index"] = index
                book_dict["book_title"] = book.text
                
                poems_list = []
                poem_looping = True
                
                # hack to let first book's epigram have a zero index
                if index == 1:
                    start_num = 0
                else:
                    start_num = 1
                # begin poem loop
                for index, poem in enumerate(book.find_next_siblings('h3'), start=start_num):
                    lines_list = []
                    poem_dict = OrderedDict()
                    poem_dict["poem_index"] = index
                    poem_dict["poem_title"] = poem.text
                    line_looping = True
                    # begin line loop
                    for index, line in enumerate(poem.find_next_siblings(), start=1):
                        line_dict = {}
                        line_dict["line_index"] = index
                        line_dict["meter"] = "none"
                        try:
                            tag_name = line.name
                        except AttributeError:
                            tag_name = ""
                        if tag_name == "p":
                            text = line.text
                            line_dict["text"] = text
                            lines_list.append(line_dict)
                        elif tag_name == "hr":
                            line_looping = False
                            poem_looping = False
                        else:
                            line_looping = False
                            # this break just puts us back into the h3 loop
                            break
                    poems_list.append(poem_dict)
                    if not poem_looping:
                        break
                book_dict["poems"]= poems_list
                books_list.append(book_dict)
            work_dict["books"] = books_list
            works_list.append(work_dict)

def get_ars(works_list):
    # article tags surround works, all other tags are unnested, boo
    for index, work in enumerate(soup.findAll('article')):
        # get the Ars
        if index==2:
            # the h1 is the title
            work_dict = OrderedDict() # otherwise the title is last which is not what we want in the JSON
            work_dict["title"] = work.find('h1').text
#             work_dict["abbreviation"] = work.attrib["n"]
            work_dict["slug"] = "ars"
            
            books_list = []
            book_looping = True
            # only h2s with this class indicate Book titles
            for index, book in enumerate(work.findAll('h2', class_='text-align-center'), start=1):
                book_dict = OrderedDict()
                book_dict["book_index"] = index
                book_dict["book_title"] = book.text
               
                poem_dict = OrderedDict()
                poem_dict["poem_index"] = 1
                poem_dict["poem_title"] = "Poem 1"
                
                poems_list = []
                lines_list = []
                line_looping = True
                anchor = book.find_next_sibling('h3')
                for index, line in enumerate(anchor.find_next_siblings('p'), start=1):
                    line_dict = {}
                    line_dict["line_index"] = index
                    line_dict["meter"] = "none"
                    try:
                        tag_name = line.name
                    except AttributeError:
                        tag_name = ""
                    if tag_name == "p":
                        text = line.text
#                         print text
                        line_dict["text"] = text
                        lines_list.append(line_dict)
                    elif tag_name == "hr":
                        line_looping = False
                        book_looping = False
                poem_dict['lines'] = lines_list
                poems_list.append(poem_dict)
                if not book_looping:
                    break
                book_dict["poems"]= poems_list
                books_list.append(book_dict)
            work_dict["books"] = books_list
            works_list.append(work_dict)

get_ars(works_list)
corpus["works"]=works_list

json_corpus = json.dumps(corpus)
print json_corpus

# structure notes
#poems have individual anchor links , so for Amores, start anchor_Toc520535255 end anchor_Toc520536672
# basic html structure -- not much nesting :/
# <article> #indicates work
# <h1 id="Amoreshome">The Amores</h1> #indicates title of work
# <h2 class="text-align-center" id="Amoresbk1">Book I</h2> #indicates book num, has class, has hr beforehand
# <h2>Contents</h2> #hanging out, does not have class
# <h3><a id="anchor_Toc520535255">His Epigram</a></h3> #actual poems have h3 at head, no surrounding tags
# <h2 class="text-align-center" id="Amoresbk2">Book II</h2>
# <h2>Contents</h2>
# <h3><a id="anchor_Toc520535255">His Epigram</a></h3>
# <h2 class="text-align-center" id="Amoresbk3">Book III</h2>
# <h2>Contents</h2>
# </article> #ends just after p of last poem
# h2 Amatoriabk1
# end of book:
# <hr />
# <h2 class="text-align-center" id="Amoresbk2">Book II</h2>


# other notes
# https://stackoverflow.com/questions/11647348/find-next-siblings-until-a-certain-one-using-beautifulsoup

{"author": [{"name": "A. S. Kline"}, {"slug": "tony-kline"}, {"full name": "Tony Kline"}, {"abbreviation": "A. Kl."}], "works": [{"title": "Ars Amatoria (The Art of Love)", "slug": "ars", "books": [{"book_index": 1, "book_title": "Book I", "poems": [{"poem_index": 1, "poem_title": "Poem 1", "lines": [{"text": "Should anyone here not know the art of love,", "line_index": 1, "meter": "none"}, {"text": "read this, and learn by reading how to love.", "line_index": 2, "meter": "none"}, {"text": "By art the boat\u2019s set gliding, with oar and sail,", "line_index": 3, "meter": "none"}, {"text": "by art the chariot\u2019s swift: love\u2019s ruled by art.", "line_index": 4, "meter": "none"}, {"text": "Automedon\u00a0was skilled with Achilles\u2019s chariot reins,", "line_index": 5, "meter": "none"}, {"text": "Tiphys\u00a0in Thessaly was steersman of the Argo,", "line_index": 6, "meter": "none"}, {"text": "Venus appointed me as guide to gentle Love:", "line_index": 7, "meter": "none"}, {"text"